In [7]:
from operator import itemgetter

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()
chain2 = ({"city": chain1, "language": itemgetter("language")}) | prompt2 | model | StrOutputParser()

chain2.invoke({"person": "Michelle Obama", "language": "portuguese"})


'O país em que a cidade de Michelle Obama, Chicago, Illinois, está localizada é os Estados Unidos da América.'

In [8]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

In [14]:
model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)

color2fruit = prompt2 | model_parser
color2country = prompt3 | model_parser

question_generator = (
    color_generator | {"fruit": color2fruit, "country": color2country} | prompt4 
)

In [16]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Marigold and the flag of India?')])

In [18]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color of a pomegranate is usually a deep red or maroon. The flag of Mexico consists of three vertical stripes in the colors green, white, and red, from left to right.')

Branching and Merging

In [22]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}.")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template("List the pros or positive aspects of {base_response}")
    | ChatOpenAI()
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template("List the cons or negative aspects of {base_response}")
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{response1}\n\nCons:\n{response2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner 
    | {
        "response1": arguments_for,
        "response2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [24]:
chain.invoke({"input": "TV shows"})

'While it is true that there are negative aspects to TV shows becoming an essential part of our daily lives, it is important to recognize that these cons can be mitigated with responsible viewing habits. It is up to individuals to manage their time, prioritize their health and well-being, and critically analyze the content they consume.\n\nBy being selective about the TV shows we watch and choosing those that offer educational value, promote positive social change, and inspire personal growth, we can maximize the benefits of this medium. It is also important to balance TV show consumption with other activities such as exercise, spending time with loved ones, and pursuing personal interests and goals.\n\nFurthermore, being aware of the potential negative influences of TV shows, such as unrealistic portrayals of reality and promotion of unhealthy behaviors, allows us to critically evaluate the content and make informed choices about what we watch.\n\nAdditionally, it is crucial for the T